# Download data

In [1]:
from getpass import getpass
import os
user = 'ahmadelsallab'
key = '6b7ffe97ff5bc0656e325b746b72fa31'

if '.kaggle' not in os.listdir('/root'):
    !mkdir ~/.kaggle
!touch /root/.kaggle/kaggle.json
!chmod 666 /root/.kaggle/kaggle.json
with open('/root/.kaggle/kaggle.json', 'w') as f:
    f.write('{"username":"%s","key":"%s"}' % (user, key))
!chmod 600 /root/.kaggle/kaggle.json


!kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification
!chmod 777 *
!ls

  0% 0.00/221k [00:00<?, ?B/s]
100% 221k/221k [00:00<00:00, 70.4MB/s]
 74% 9.00M/12.1M [00:00<00:00, 23.3MB/s]
100% 12.1M/12.1M [00:00<00:00, 30.6MB/s]
 98% 267M/273M [00:07<00:00, 34.0MB/s]
100% 273M/273M [00:07<00:00, 38.5MB/s]
sample_data  sample_submission.csv.zip	test.csv.zip  train.csv.zip


In [2]:
!unzip sample_submission.csv.zip
!unzip test.csv.zip
!unzip train.csv.zip
!ls

Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   
Archive:  test.csv.zip
  inflating: test.csv                
Archive:  train.csv.zip
  inflating: train.csv               
sample_data	       sample_submission.csv.zip  test.csv.zip	train.csv.zip
sample_submission.csv  test.csv			  train.csv


# Explore data samples

In [0]:
trn_limit = 20000
tst_limit = 20000

In [4]:
import pandas as pd
train_df = pd.read_csv('train.csv')[:trn_limit]
train_df.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:41.987077+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:42.870083+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:45.222647+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:47.601894+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:48.488476+00,2,NaN,2006,rejected,0,0,0,1,0,0.0,4,47


In [5]:
import pandas as pd
test_df = pd.read_csv('test.csv')[:tst_limit]
test_df.head()

,id,comment_text
0,7000000,Jeff Sessions is another one of Trump's Orwell...
1,7000001,I actually inspected the infrastructure on Gra...
2,7000002,No it won't . That's just wishful thinking on ...
3,7000003,Instead of wringing our hands and nibbling the...
4,7000004,how many of you commenters have garbage piled ...


# Data pre-processing

In [0]:
import numpy as np

In [0]:
trn_texts = list(train_df['comment_text'])
tst_texts = list(test_df['comment_text'])
texts = trn_texts + tst_texts

In [8]:
from keras.preprocessing.text import Tokenizer, text_to_word_sequence

tokenizer = Tokenizer()


Using TensorFlow backend.


In [0]:
tokenizer.fit_on_texts(texts)


In [0]:
del texts
del tst_texts
del test_df

In [0]:
binary_trn_texts = tokenizer.texts_to_sequences(trn_texts)

In [28]:

# TODO: Calc max seqlen
maxlen = max([len(s) for s in binary_trn_texts])
maxlen
#idx = np.argmax([len(s) for s in binary_trn_texts])
#trn_texts[idx]

317

In [0]:
from keras.preprocessing.sequence import pad_sequences

x_train = np.array(pad_sequences(binary_trn_texts, maxlen=maxlen, padding='post'))


In [0]:
del binary_trn_texts

In [32]:
x_train.shape

(20000, 317)

In [30]:
x_train[10]

array([877,   8, 585, 153,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [41]:
y_train = np.array(list(train_df['target']))
y_train.shape

(20000,)

In [42]:
y_train[:10]

array([0.        , 0.        , 0.        , 0.        , 0.89361702,
       0.66666667, 0.45762712, 0.        , 0.        , 0.        ])

In [17]:

# Shuffle
'''
np.random.seed(42)
trn_idx = np.random.permutation(len(trn_texts))

x_train = x_train[trn_idx]
y_train = y_train[trn_idx]
'''


'\nnp.random.seed(42)\ntrn_idx = np.random.permutation(len(trn_texts))\n\nx_train = x_train[trn_idx]\ny_train = y_train[trn_idx]\n'

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42, shuffle=True)

# Model

In [0]:
model_name = 'cnn_lstm_text_only'

In [59]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D

vocab_size = len(tokenizer.word_index)
embedding_size = 200

# Convolution
kernel_size = 3
filters = 256
pool_size = 3

# LSTM
lstm_output_size = 1000




model = Sequential()
model.add(Embedding(vocab_size, embedding_size, mask_zero=False, input_length=maxlen))
#model.add(Dropout(0.25))
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=pool_size))
model.add(LSTM(lstm_output_size))
model.add(Dense(500))
model.add(Dense(200))
model.add(Dense(100))
model.add(Dense(1, activation='sigmoid'))

model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 317, 200)          11241000  
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 315, 256)          153856    
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 105, 256)          0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 1000)              5028000   
_________________________________________________________________
dense_29 (Dense)             (None, 500)               500500    
_________________________________________________________________
dense_30 (Dense)             (None, 200)               100200    
_________________________________________________________________
dense_31 (Dense)             (None, 100)               20100     
__________

In [37]:
# Mount GDrive
from google.colab import drive
drive.mount('/content/gdrive')
gdrive_path = 'gdrive/My Drive/Colab Notebooks/JIGSAW/dat'

from keras.callbacks import ModelCheckpoint, Callback
filepath = os.path.join(gdrive_path, 'jigsaw_' + model_name + '.h5')
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

callbacks_lst = [checkpoint]


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [61]:
from keras import optimizers
# Optimizer
lr = 0.001
opt = optimizers.Adam(lr=lr)
model.compile(loss='mse',
              optimizer=opt,
              metrics=['accuracy'])

print('Train...')
# Training
batch_size = 256
epochs = 20
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,          
          validation_data=(x_val, y_val),
          #validation_split=0.2,
          callbacks=callbacks_lst)

Train...
Train on 18000 samples, validate on 2000 samples
Epoch 1/20
18000/18000 [==============================] - 31s 2ms/step - loss: 0.0389 - acc: 0.7469 - val_loss: 0.0449 - val_acc: 0.7220

Epoch 00001: val_acc did not improve from 0.72200
Epoch 2/20
18000/18000 [==============================] - 27s 2ms/step - loss: 0.0389 - acc: 0.7469 - val_loss: 0.0449 - val_acc: 0.7220

Epoch 00002: val_acc did not improve from 0.72200
Epoch 3/20
18000/18000 [==============================] - 27s 2ms/step - loss: 0.0389 - acc: 0.7469 - val_loss: 0.0449 - val_acc: 0.7220

Epoch 00003: val_acc did not improve from 0.72200
Epoch 4/20
18000/18000 [==============================] - 27s 2ms/step - loss: 0.0389 - acc: 0.7469 - val_loss: 0.0449 - val_acc: 0.7220

Epoch 00004: val_acc did not improve from 0.72200
Epoch 5/20
18000/18000 [==============================] - 27s 2ms/step - loss: 0.0389 - acc: 0.7469 - val_loss: 0.0449 - val_acc: 0.7220

Epoch 00005: val_acc did not improve from 0.72200
Ep

KeyboardInterrupt: ignored

In [23]:
from sklearn.metrics import roc_auc_score, auc
# y_val is continous, but according to the competetion, the thresh is at 0.5, so we can convert to binary
# The *1 is to convert False/True into 0/1
#fpr, tpr, thresh  = roc_auc_score(np.array((y_val>0.5)*1), model.predict(x_val))
#val_auc = auc(fpr, tpr)
val_auc = roc_auc_score(np.array((y_val>0.5)*1), model.predict(x_val))
val_auc

0.5

# Prepare test data

In [0]:
import pandas as pd
test_df = pd.read_csv('test.csv')
test_df.head()
tst_texts = list(test_df['comment_text'])
binary_tst_texts = tokenizer.texts_to_sequences(tst_texts)
x_test = np.array(pad_sequences(binary_tst_texts, maxlen=maxlen))

In [0]:
y_test = model.predict(x_test)

sub_df = pd.DataFrame({'id': test_df['id'], 'prediction':list(y_test)})

sub_df.to_csv('submission_' + model_name + '.csv')
                      
